#  Notebook 1: Data Processing

In this notebook I want to complete the following things:

> *Determine ground-truth labels from the metadata and merge the two data dumps for analysis (expected content in hand-in: code)*

#### Methodology
* loading data
* cleaning/preprocesing data
* combining data 
* saving data for analysis later

#### Project Structure
```
/venv
/tractable_ds_excercise_data
/data_processing.ipynb (this notebook)
```

#### DataPipeline description:
* Remove the unncessary fields (e.g.make,model,year,poi,line_num, part_price,labour_amt)
* Collate classifier scores into vector representation
* Collate metadata scores into vector representations

Merged Table Structure:

| Field | Description |
| :--- | :----------- |
| claim_id | Claim id (int) |
| part | Name of car body part (str) |
| urr_score | Score from classifier (float) |
| set | Train/Validation/Test split (int) |
| operation | Operation conducted (undamaged/repair/replace) |



#### Data Findings

Data Breakdown (Train/Validation/Test):

|  | Undamaged | Repair | Replace | Total |
| :--- | --------- | ------ | ------- | ----- |
| Train| 639766 | 111713 | 48401 | 799880
| Validation | 80021 | 13724 | 6035 |99780
| Test| 80386 | 14028 | 5926 |100340


Breakdown of Repair and Replace by parts:


### Import Data

In [9]:
# Requirements
!pip install pandas

DATA_FILE = './tractable_ds_excercise_data'

In [125]:
# Lets check classifier data
import pandas as pd

c_df = pd.read_csv(DATA_FILE + '/classifier_output.csv')
c_df

,claim_id,part,urr_score,set
0,0,fbumper,0.464102,0
1,0,bbumper,0.680342,0
2,0,bldoor,NaN,0
3,0,brdoor,NaN,0
4,0,frdoor,NaN,0
...,...,...,...,...
999995,99999,fldoor,0.237536,0
999996,99999,frwing,NaN,0
999997,99999,flwing,NaN,0
999998,99999,brwing,0.429212,0


In [143]:
# Lets check metadata
import os

df_list = [pd.read_csv(DATA_FILE + '/metadata'+ '/'+ file) for file in os.listdir(DATA_FILE +'/metadata')]
m_df = pd.concat(df_list)

# lets also remove all unneccessary fields
m_df = m_df.drop(columns=['make','model','year', 'poi', 'line_num', 'part_price', 'labour_amt', 'Unnamed: 0'])
m_df

,claim_id,part,operation
0,7535.0,fbumper,replace
1,2643.0,bbumper,repair
2,50303.0,blwing,repair
3,73273.0,frwing,repair
4,53560.0,fldoor,repair
...,...,...,...
4995,77280.0,brdoor,repair
4996,32276.0,bbumper,replace
4997,4165.0,flwing,repair
4998,50613.0,bbumper,repair


### Clean / Merge Data

In [127]:
from typing import Tuple

def get_classifier_and_metadata(claim_id: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    return c_df[c_df['claim_id']==claim_id], m_df[m_df['claim_id']==claim_id]

get_classifier_and_metadata(20)

(     claim_id     part  urr_score  set
 200        20  fbumper   0.269102    0
 201        20  bbumper   0.447384    0
 202        20   bldoor   0.412994    0
 203        20   brdoor   0.331167    0
 204        20   frdoor        NaN    0
 205        20   fldoor   0.485910    0
 206        20   frwing        NaN    0
 207        20   flwing   0.368965    0
 208        20   brwing   0.301396    0
 209        20   blwing   0.575618    0,
       claim_id     part operation
 615       20.0   blwing    repair
 4486      20.0  bbumper    repair)

In [148]:
merged_pd = pd.merge(c_df, m_df, on=['claim_id', 'part'], how='left')

# also where we have empty values (e.g no claim is made, set these values to 0)
merged_pd['operation'] = merged_pd['operation'].fillna('undamaged')

In [149]:
# lets just sanity check our merged PD with the both original tables
claim_id = 5430

merged_pd[merged_pd['claim_id']==claim_id], get_classifier_and_metadata(claim_id)

(       claim_id     part  urr_score  set  operation
 54300      5430  fbumper   0.380594    1     repair
 54301      5430  bbumper   0.542645    1  undamaged
 54302      5430   bldoor   0.384501    1  undamaged
 54303      5430   brdoor   0.150392    1  undamaged
 54304      5430   frdoor   0.095429    1  undamaged
 54305      5430   fldoor   0.662270    1    replace
 54306      5430   frwing        NaN    1  undamaged
 54307      5430   flwing   0.537756    1    replace
 54308      5430   brwing   0.405801    1  undamaged
 54309      5430   blwing   0.731776    1     repair,
 (       claim_id     part  urr_score  set
  54300      5430  fbumper   0.380594    1
  54301      5430  bbumper   0.542645    1
  54302      5430   bldoor   0.384501    1
  54303      5430   brdoor   0.150392    1
  54304      5430   frdoor   0.095429    1
  54305      5430   fldoor   0.662270    1
  54306      5430   frwing        NaN    1
  54307      5430   flwing   0.537756    1
  54308      5430   brwing   

### Analyse the performance
* Produce ROC graphs for classifiying normal vs replace, normal vs repair and repair vs replace
* Calculate AUC for each of the above graphs
* Be clear to only use test set for calculations

In [150]:
# lets firstly clean up operations data to be integer
m_df['claim_id'] = [int(value) for value in m_df['claim_id']]

def operation2code(x):
    if x == 'undamaged':
        return 0
    elif x == 'repair':
        return 1
    elif x == 'replace':
        return 2
    else:
        raise ValueError(f"recieved operation: {x}")

merged_pd['operation'] = merged_pd['operation'].apply(operation2code)

In [154]:
merged_pd[merged_pd['set']==2]
merged_pd.to_csv("./combined_data.csv")

In [173]:
# additional data analysis
for set_idx in range(3):
    print(len(merged_pd.query(f'set=={set_idx}')))
    for status_idx in range(3):
        print(f"Set: {set_idx}, Status: {status_idx}: {len(merged_pd.query(f'set=={set_idx} & operation=={status_idx}'))}")

799880
Set: 0, Status: 0: 639766
Set: 0, Status: 1: 111713
Set: 0, Status: 2: 48401
99780
Set: 1, Status: 0: 80021
Set: 1, Status: 1: 13724
Set: 1, Status: 2: 6035
100340
Set: 2, Status: 0: 80386
Set: 2, Status: 1: 14028
Set: 2, Status: 2: 5926


In [ ]:
status_idx = 1
for set_idx in range(3):
    print(f"Set: {set_idx}, Status: {status_idx}: {len(merged_pd.query(f'set=={set_idx} & operation=={status_idx}'))}")